In [1]:
import pandas as pd
import sys
import os
import time
import random

In [2]:
# Import libraries
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# Time imports
import random
import datetime
import time

# Other imports
import re as re
import pandas as pd
import os
import sys
import numpy as np

import pymysql
import sqlalchemy as alch
from getpass import getpass

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
mercadona_user = os.getenv("mercadona_user")
mercadona_password = os.getenv("mercadona_password")

In [110]:
def get_purchase_history(zip, mercadona_user, mercadona_password, headless=True):
    """
    """
    # create a ChromeOptions object
    options = Options()

    # add the headless argument if passed
    if headless:
        options.add_argument('--headless')

    # specify the path to your web driver
    driver = webdriver.Chrome(options=options)

    # navigate to the login page
    driver.get('https://www.mercadona.es/')

    # enter the postal code and submit it
    postal_code = driver.find_element(By.CSS_SELECTOR, 'input[aria-label="Código postal"]').send_keys(zip)
    submit_button = driver.find_element(By.CSS_SELECTOR, 'input.postal-code-form__button').click()

    # Accept cookies
    accept_button = driver.find_element(By.XPATH, "//button[contains(text(),'Aceptar todas')]").click()

   # wait for the page to load
    driver.implicitly_wait(10)

    # click the user dropdown and click the "Identifícate" button
    drop_down = driver.find_element(By.CSS_SELECTOR, 'button[class="drop-down__trigger"]')
    drop_down.click()
    identificate = driver.find_element(By.CSS_SELECTOR,'a[href="/?authenticate-user="]').click()

    #<a class="btn btn--primary btn--default" href="/?authenticate-user="><span class="button__text">Identifícate</span></a>
   
    # enter the email address
    email = driver.find_element(By.CSS_SELECTOR, 'input[name="email"]')
    email.send_keys(mercadona_user)

    # click the "Siguiente" button
    next_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
    next_button.click()

    # enter the password
    password = driver.find_element(By.CSS_SELECTOR, 'input[name="password"]')
    password.send_keys(mercadona_password)

    # click the "Entrar" button
    entrar_button = driver.find_element(By.CSS_SELECTOR, 'button[data-test="do-login"]')
    entrar_button.click()

    # Wait up to 10 seconds for the element to become visible
    personal_section = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located(
            (By.CLASS_NAME, 'account__user-name')
        )
    )
    personal_section.click()
  
    # Open "Mis pedidos"
    pedidos = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located(
            (By.CSS_SELECTOR, 'a[href="/user-area/orders"]')
        )
    )
    pedidos.click()

    # Get all order numbers:
    order_nums = WebDriverWait(driver, 10).until(
        EC.visibility_of_all_elements_located(
            (By.CSS_SELECTOR, 'span[class="order-cell__id footnote1-r"]')
        )
    )
    
    list_of_orders = []
    for pedido in order_nums:
        list_of_orders.append(pedido.text.split(' ')[1])

    pedidos_to_return = pd.DataFrame({})

    for order_text in list_of_orders:
        driver.get(f"https://tienda.mercadona.es/user-area/orders/{order_text}?products")

        order_details = {}
        # Get all products
        products = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located(
                (By.CSS_SELECTOR, 'p[class="order-product-cell__name subhead1-r"]')
            )
        )
        products_list = []
        for product in products:
            products_list.append(product.text)
        order_details["product"] = products_list

        # Get all units
        units = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located(
                (By.CSS_SELECTOR, 'span[class="order-product-cell__prepared-units subhead1-r"]')
            )
        )
        units_list = []
        for unit in units:
            units_list.append(int(unit.text.split(' ')[0]))
        order_details["units"] = units_list
        
        # Get all prices
        prices = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located(
                (By.CSS_SELECTOR, 'p[class="order-product-cell__price subhead1-r"]')
            )
        )
        prices_list = []
        for price in prices:
            prices_list.append(float(price.text.split(' ')[0].replace(',','.')))
        order_details["price"] = prices_list

        order_details_df = pd.DataFrame(order_details)
        order_details_df = order_details_df.assign(order_number=order_text)
        pedidos_to_return = pd.concat([pedidos_to_return,order_details_df],ignore_index=True)

    print("success")
    return pedidos_to_return

In [111]:
orders = get_purchase_history('08012', mercadona_user, mercadona_password, headless=False)

success


In [112]:
orders

,product,units,price,order_number
0,"Salsa de tomate, albahaca y orégano Hacendado",1,1.20,13945376
1,Tomate pera,2,0.60,13945376
2,Atún en aceite de girasol Hacendado,2,13.60,13945376
3,Café molido natural Hacendado,1,3.20,13945376
4,Hummus de garbanzos Hacendado con pimiento del...,1,1.45,13945376
...,...,...,...,...
637,Pechugas enteras de pollo,1,4.32,12793662
638,Pechugas enteras de pollo,1,4.40,12793662
639,Medallones pechuga de pollo marinado,1,4.32,12793662
640,Aguacates,1,2.90,12793662


In [114]:
sum(orders["product"].str.contains(","))

11

In [116]:
orders.to_csv('order_history.csv', sep="~")